# Reshape Datasets

### Hello, everyone, welcome to my video. In this video, I will introduce a very important function in InMemoryDatasets package, which is `transpose` function. 

## Introduction

### As data scientists, we know that data does not always come to us with the most desirable format that’s ready for analysis or visualization. Therefore, we need to reshape it as we want it to be for subsequent operations

### In InMemoryDatasets package, the reshaping of a data set is done by the `transpose` function. In the simple case, you can think about the data set as a matrix, and `transpose` simply flips it over its diagonal by switching the row and column indices of it.

## Simple Case

### Transpose means switching the row and column indices of a matrix, and in InMemoryDatasets when we select a set of columns, we practically have a matrix shape array of data, thus, its transposition means switching the row and column indices of it. Thus, in the simplest form, the syntax of the `transpose` function is `transpose(ds, cols)`, where `ds` is the input data set and `cols` is any kind of column selector which specifies the selected columns for transposing. Since each column of a data set has also a name, `transpose` creates a new column in the output data set which shows those names from the input data set.

### We have an example:

In [4]:
using InMemoryDatasets

In [5]:
ds1 = Dataset(x1 = [1,2,3,4], x2 = [1,4,9,16])

4×2 Dataset
 Row │ x1        x2       
     │ identity  identity 
     │ Int64?    Int64?   
─────┼────────────────────
   1 │        1         1
   2 │        2         4
   3 │        3         9
   4 │        4        16

### And then we just use transpose to flips it over

In [6]:
transpose(ds1, 1:2)

2×5 Dataset
 Row │ _variables_  _c1       _c2       _c3       _c4      
     │ identity     identity  identity  identity  identity 
     │ String?      Int64?    Int64?    Int64?    Int64?   
─────┼─────────────────────────────────────────────────────
   1 │ x1                  1         2         3         4
   2 │ x2                  1         4         9        16

### We can see the rows of the orignal data set haven't got names, thus, InMemoryDatasets uses automatic name generation to produce the names for the transposed columns of the output data set.  User can supply a custom function for generating the names of the transposed columns by using the keyword argument `renamecolid`, which by default is set as` i -> "_c" * string(i)` where `i` is the sequence of columns.

### We also have an example here to show how to use `renamecolid`

In [8]:
transpose(ds1, [:x1,:x2], renamecolid = x -> "_COLUMN_" * string(x))

2×5 Dataset
 Row │ _variables_  _COLUMN_1  _COLUMN_2  _COLUMN_3  _COLUMN_4 
     │ identity     identity   identity   identity   identity  
     │ String?      Int64?     Int64?     Int64?     Int64?    
─────┼─────────────────────────────────────────────────────────
   1 │ x1                   1          2          3          4
   2 │ x2                   1          4          9         16

###  User can also set the `id` keyword argument to a column in the input data set when the values of the column can be used as the names for the transposed columns in the output data set. 

### As for example, we need to insert a new column to the orignal dataset

In [9]:
insertcols!(ds1,1,:id => ["r1", "r2", "r3", "r4"])

4×3 Dataset
 Row │ id        x1        x2       
     │ identity  identity  identity 
     │ String?   Int64?    Int64?   
─────┼──────────────────────────────
   1 │ r1               1         1
   2 │ r2               2         4
   3 │ r3               3         9
   4 │ r4               4        16

### `insertcols!` means insert a column into a data set in place and return the update Dataset. We also can use the `?insertcols!` in Julia to find some useful things. 

In [10]:
?insertcols!

search: insertcols!



```
insertcols!(ds::Dataset[, col], (name=>val)::Pair...;
            makeunique::Bool=false)
```

Insert a column into a data set in place. Return the updated `Dataset`. If `col` is omitted it is set to `ncol(ds)+1` (the column is inserted as the last column).

# Arguments

  * `ds` : the Dataset to which we want to add columns
  * `col` : a position at which we want to insert a column, passed as an integer or a column name (a string or a `Symbol`); the column selected with `col` and columns following it are shifted to the right in `ds` after the operation
  * `name` : the name of the new column
  * `val` : an `AbstractVector` giving the contents of the new column or a value of any type other than `AbstractArray` which will be repeated to fill a new vector; As a particular rule a values stored in a `Ref` or a `0`-dimensional `AbstractArray` are unwrapped and treated in the same way.
  * `makeunique` : Defines what to do if `name` already exists in `ds`; if it is `false` an error will be thrown; if it is `true` a new unique name will be generated by adding a suffix

If `val` is an `AbstractRange` then the result of `collect(val)` is inserted.

# Examples

```jldoctest
julia> ds = Dataset(a=1:3)
3×1 Dataset
 Row │ a
     │ identity
     │ Int64?
─────┼──────────
   1 │        1
   2 │        2
   3 │        3

julia> insertcols!(ds, 1, :b => 'a':'c')
3×2 Dataset
 Row │ b         a
     │ identity  identity
     │ Char?     Int64?
─────┼────────────────────
   1 │ a                1
   2 │ b                2
   3 │ c                3

julia> insertcols!(ds, 2, :c => 2:4, :c => 3:5, makeunique=true)
3×4 Dataset
 Row │ b         c         c_1       a
     │ identity  identity  identity  identity
     │ Char?     Int64?    Int64?    Int64?
─────┼────────────────────────────────────────
   1 │ a                2         3         1
   2 │ b                3         4         2
   3 │ c                4         5         3

```


### We can see Julia give us a detailed explanation about `insertcols!`, including its syntax, the explanation of each keyword, and many examples.

### Then we can use `id` to name our columns in the output data set.

In [11]:
transpose(ds1, [:x1,:x2], id = :id)

2×5 Dataset
 Row │ _variables_  r1        r2        r3        r4       
     │ identity     identity  identity  identity  identity 
     │ String?      Int64?    Int64?    Int64?    Int64?   
─────┼─────────────────────────────────────────────────────
   1 │ x1                  1         2         3         4
   2 │ x2                  1         4         9        16

## `transpose` of grouped data sets
###  The key feature that makes `transpose` versatile and powerful is its ability to do the simple transposing within each group of observations created by `groupby!`, `groupby` or `gatherby`. We will discuss the combination between `transpose` and these three functions below.

### Before we start this part, maybe we should introduce a little bit about `groupby` and `gatherby`, this is a big and useful part of data manipulation, I will try my best to give you a comprehensive understanding, but if you still have something that can't understand, I suggest you go to the official document in GitHub, I also put the website in the introduction of this video.

## groupby() and gatherby()

### InMemoryDatasets uses two approaches to group observations: sorting, and hashing. In sorting approach, it sorts the data set based on given columns and finds the starts and ends of each group based on the sorted values. In hashing approach, it uses a hybrid algorithm to group observations. Each of these approaches has some advantages over the other one and for any particular problem one of them might be more suitable than the other one.

### The main functions for grouping observations based on sorting approach are `groupby!` and `groupby`. The `groupby!` function replaces the original data set with the sorted one and attaches a meta information about the grouping orders to the replaced data set, on the other hand, the `groupby` function performs the sorting phase, however, it creates a view of the main data set where the meta information is attached to it. The output of `groupby` is basically a view of the sorted data set.

### Let's have an example here, assume we have a data set, 

In [12]:
ds2 = Dataset(g = [1,2,1,2,1,2], x = [12.0,12.3,11.0,13.0,15.0,13.2])

6×2 Dataset
 Row │ g         x        
     │ identity  identity 
     │ Int64?    Float64? 
─────┼────────────────────
   1 │        1      12.0
   2 │        2      12.3
   3 │        1      11.0
   4 │        2      13.0
   5 │        1      15.0
   6 │        2      13.2

### And we want to classify the dataset according to the information in the first column, we have

In [13]:
groupby(ds2,1)

6×2 View of Grouped Dataset, Grouped by: g
 g         x        
 identity  identity 
 Int64?    Float64? 
────────────────────
        1      12.0
        1      11.0
        1      15.0
        2      12.3
        2      13.0
        2      13.2

### And if we output the orignal dataset, 

In [14]:
ds2

6×2 Dataset
 Row │ g         x        
     │ identity  identity 
     │ Int64?    Float64? 
─────┼────────────────────
   1 │        1      12.0
   2 │        2      12.3
   3 │        1      11.0
   4 │        2      13.0
   5 │        1      15.0
   6 │        2      13.2

### We can see the ds2 is untouched. If we use groupby!,

In [15]:
groupby!(ds2, 1)

6×2 Grouped Dataset with 2 groups
Grouped by: g
 Row │ g         x        
     │ identity  identity 
     │ Int64?    Float64? 
─────┼────────────────────
   1 │        1      12.0
   2 │        1      11.0
   3 │        1      15.0
   4 │        2      12.3
   5 │        2      13.0
   6 │        2      13.2

### And also we output the orignal dataset,

In [16]:
ds2

6×2 Grouped Dataset with 2 groups
Grouped by: g
 Row │ g         x        
     │ identity  identity 
     │ Int64?    Float64? 
─────┼────────────────────
   1 │        1      12.0
   2 │        1      11.0
   3 │        1      15.0
   4 │        2      12.3
   5 │        2      13.0
   6 │        2      13.2

### We can see the orignal dataset has been replaced with its grouped version.

### The difference between `groupby` and `groupby!` is basically because of the exclamatory mark. In most cases in Julia, if there is an exclamatory mark after the function, which means this function will directly update the original dataset, and if there is not, the function will only return a modified copy but not change the original dataset.

### As for `gatherby`, the `gatherby` function uses the hashing approach to group observations based on a set of columns. Unlike `groupby` or `groupby!`, the `gatherby` function doesn't sort the data set, instead, it uses the hybrid algorithm to group observations. `gatherby` can be particularly useful when sorting is computationally expensive. Another benefit of `gatherby` is that, by default, it keeps the order of observations in each group the same as their appearance in the original data set.

### The syntax for using the `gatherby` function is `gatherby(ds, cols)` where `ds` is the data set and `cols` is any column selector which indicates the columns which are going to be used in gathering.

### We also have an example for `gatherby`:

### First we need a dataset, 

In [17]:
ds3 = Dataset(grp = [1,2,3,3,1,3,2,1], x = [true, false, true, true, true,true,false,false])

8×2 Dataset
 Row │ grp       x        
     │ identity  identity 
     │ Int64?    Bool?    
─────┼────────────────────
   1 │        1      true
   2 │        2     false
   3 │        3      true
   4 │        3      true
   5 │        1      true
   6 │        3      true
   7 │        2     false
   8 │        1     false

In [18]:
gatherby(ds3,:x)

8×2 View of GatherBy Dataset, Gathered by: x
 grp       x        
 identity  identity 
 Int64?    Bool?    
────────────────────
        1      true
        3      true
        3      true
        1      true
        3      true
        2     false
        2     false
        1     false

### As mentioned before, the result of `gatherby` is stable, however, when this stability is not needed and there are many groups in the data set, passing `stable = false` improves the performance by sacrificing the stability.

### Through the above, I believe you have a comprehensive understanding of `groupby` and `gatherby`. Let's continue our `transpose` learning.

### When the first argument of the `transpose` function is a grouped data set - created by `groupby!`, `groupby`, or `gatherby` - `transpose` does the simple transposing within each group of observations. Thus, the transposition of a grouped data set can be viewed as transposing the matrix shape data values which are created for each group of observations. Since the size of transposed columns within each group can be different, `transpose` pads them with `missing` values to overcome this problem. The `missing` padding can be replaced by any other values which passed to default keyword argument of the function.

### We still begin with an example, assume we have a dataset,

In [19]:
ds4 = Dataset(group = repeat(1:3, inner = 2),
                                    b = repeat(1:2, inner = 3),
                                    c = repeat(1:1, inner = 6),
                                    d = repeat(1:6, inner = 1),
                                    e = string.('a':'f'))

6×5 Dataset
 Row │ group     b         c         d         e        
     │ identity  identity  identity  identity  identity 
     │ Int64?    Int64?    Int64?    Int64?    String?  
─────┼──────────────────────────────────────────────────
   1 │        1         1         1         1  a
   2 │        1         1         1         2  b
   3 │        2         1         1         3  c
   4 │        2         2         1         4  d
   5 │        3         2         1         5  e
   6 │        3         2         1         6  f

### Assume we want to transpose the grouped dataset based on `group` column and we only want to use the integer data. 

In [20]:
transpose(groupby(ds4,:group),2:4)

9×4 Dataset
 Row │ group     _variables_  _c1       _c2      
     │ identity  identity     identity  identity 
     │ Int64?    String?      Int64?    Int64?   
─────┼───────────────────────────────────────────
   1 │        1  b                   1         1
   2 │        1  c                   1         1
   3 │        1  d                   1         2
   4 │        2  b                   1         2
   5 │        2  c                   1         1
   6 │        2  d                   3         4
   7 │        3  b                   2         2
   8 │        3  c                   1         1
   9 │        3  d                   5         6

### Also when we want to use column e for the name of the transposed dataset, we can use `id` keyword.

In [21]:
transpose(groupby(ds4,:group),2:4,id = :e)

9×8 Dataset
 Row │ group     _variables_  a         b         c         d         e        ⋯
     │ identity  identity     identity  identity  identity  identity  identity ⋯
     │ Int64?    String?      Int64?    Int64?    Int64?    Int64?    Int64?   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │        1  b                   1         1   missing   missing   missing ⋯
   2 │        1  c                   1         1   missing   missing   missing
   3 │        1  d                   1         2   missing   missing   missing
   4 │        2  b             missing   missing         1         2   missing
   5 │        2  c             missing   missing         1         1   missing ⋯
   6 │        2  d             missing   missing         3         4   missing
   7 │        3  b             missing   missing   missing   missing         2
   8 │        3  c             missing   missing   missing   missing         1
   9 │        3  d             missing   missing   missing   missing         5 ⋯
                                                                1 column omitted

### As you can see, there are many `missing` because the size of transposed columns within each group can be different, and assume we want to use `None` to replace it.

In [23]:
transpose(groupby(ds4,:group),2:4,id = :e, default = "None")

9×8 Dataset
 Row │ group     _variables_  a         b         c         d         e        ⋯
     │ identity  identity     identity  identity  identity  identity  identity ⋯
     │ Int64?    String?      Any       Any       Any       Any       Any      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │        1  b            1         1         None      None      None     ⋯
   2 │        1  c            1         1         None      None      None
   3 │        1  d            1         2         None      None      None
   4 │        2  b            None      None      1         2         None
   5 │        2  c            None      None      1         1         None     ⋯
   6 │        2  d            None      None      3         4         None
   7 │        3  b            None      None      None      None      2
   8 │        3  c            None      None      None      None      1
   9 │        3  d            None      None      None      None      5        ⋯
                                                                1 column omitted

## Advanced options

### The column selector of the `transpose` function can be also a `Tuple` of column selectors. In this case, InMemoryDatasets does the transposition for each element of the tuple and then horizontally concatenates the output data sets to create a single data set. This provides extra flexibility to the user for reshaping a data set. By default, the `variable_name` is set to `nothing`, when `Tuple` of column selectors is passed as the argument, however, we can supply different names for each element of the `Tuple`.

### Since the column names for the output data set can be the same for all elements of the tuple, `transpose` automatically modifies them to make them unique. Nevertheless, by passing `renamecolid`, we can customise the column names.

### It seems a little abstract. Let's use an example to understand it, assume we have a dataset

In [24]:
ds5 = Dataset([[1, 1, 1, 2, 2, 2],
                        ["foo", "bar", "monty", "foo", "bar", "monty"],
                        ["a", "b", "c", "d", "e", "f"],
                        [1, 2, 3, 4, 5, 6]], [:g, :key, :foo, :bar])

6×4 Dataset
 Row │ g         key       foo       bar      
     │ identity  identity  identity  identity 
     │ Int64?    String?   String?   Int64?   
─────┼────────────────────────────────────────
   1 │        1  foo       a                1
   2 │        1  bar       b                2
   3 │        1  monty     c                3
   4 │        2  foo       d                4
   5 │        2  bar       e                5
   6 │        2  monty     f                6

### First, we just group the data set based on column g

In [25]:
groupby(ds5,:g)

6×4 View of Grouped Dataset, Grouped by: g
 g         key       foo       bar      
 identity  identity  identity  identity 
 Int64?    String?   String?   Int64?   
────────────────────────────────────────
        1  foo       a                1
        1  bar       b                2
        1  monty     c                3
        2  foo       d                4
        2  bar       e                5
        2  monty     f                6

### And then we transpose it, and use column key for the column name.

In [26]:
transpose(groupby(ds5,:g),3:4,id = :key)

4×5 Dataset
 Row │ g         _variables_  foo       bar       monty    
     │ identity  identity     identity  identity  identity 
     │ Int64?    String?      Any       Any       Any      
─────┼─────────────────────────────────────────────────────
   1 │        1  foo          a         b         c
   2 │        1  bar          1         2         3
   3 │        2  foo          d         e         f
   4 │        2  bar          4         5         6

### Then, as I mentioned before, we use a tuple as the input of the column selector.

In [27]:
transpose(groupby(ds5,:g),(:foo,:bar),id = :key)

2×7 Dataset
 Row │ g         foo       bar       monty     foo_1     bar_1     monty_1  
     │ identity  identity  identity  identity  identity  identity  identity 
     │ Int64?    String?   String?   String?   Int64?    Int64?    Int64?   
─────┼──────────────────────────────────────────────────────────────────────
   1 │        1  a         b         c                1         2         3
   2 │        2  d         e         f                4         5         6

### From the result, the `transpose` horizontally concatenates the output data sets to create a single data set.

### The `renamecolid` function can also get access to the variable names from the input data set as the second argument. This can be used to generate even more customised column names for the output data set.

### Let's use the example above to explain this:

In [28]:
transpose(groupby(ds5,:g),(:foo,:bar), id = :key, renamecolid = (x,y) -> string(x,"_",y))

2×7 Dataset
 Row │ g         foo_["foo"]  bar_["foo"]  monty_["foo"]  foo_["bar"]  bar_["b ⋯
     │ identity  identity     identity     identity       identity     identit ⋯
     │ Int64?    String?      String?      String?        Int64?       Int64?  ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │        1  a            b            c                        1          ⋯
   2 │        2  d            e            f                        4
                                                               2 columns omitted

### From the result, we can see the name of each column has been changed through `renamecolid`.

### From the above explanation, I believe you can have a comprehensive understanding of this function. If you still have doubts about it, you can find relevant content on InMemoryDatasets official documents at any time. Thank you for watching this video, see you next time.